<a href="https://colab.research.google.com/github/PabloMendieta03/UFV-VisualizacionDeDatos/blob/main/Clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

Python 3.10.12


# Ejercicio de histórico de ventas: Planificación

## Carga de la Base de Datos

In [ ]:
# Instalación de librerías
!pip install dash

In [ ]:
# Importación de librerías
import pandas as pd
from google.colab import drive
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

In [ ]:
drive.mount('/content/drive')

# Especicicamos la ruta del archivo CSV en Google Drive
ruta_archivo_drive = '/content/drive/MyDrive/datos_ejercicio_ventas.csv'

# Cargamos el archivo CSV en un DataFrame de pandas
data = pd.read_csv(ruta_archivo_drive)

# mostrar los datos
display(data)

Mounted at /content/drive


,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,7.543562e+05
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,5.600306e+05
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,8.850198e+04
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,3.632245e+05
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,3.961761e+05
...,...,...,...,...,...,...,...,...
18661,Great Britain,Pepsi Regular (L3),2024,2,AI_forecast,AI_P10F,2023.0,1.313511e+06
18662,Hungary,Pepsi Regular (L3),2024,7,AI_forecast,AI_P07F,2023.0,1.314395e+06
18663,Norway,7up (L3),2024,1,AI_forecast,AI_P05F,2023.0,0.000000e+00
18664,Portugal,Lipton (L3),2024,3,AI_forecast,AI_P02F,2023.0,5.330634e+05


## Análisis de los Datos

### **Variable 'SCENARIO'**



In [ ]:
# Obtener las categorías únicas en 'SCENARIO'
categorias = data['SCENARIO'].unique()

# Mostrar las categorías
print("Categorías en 'SCENARIO':", categorias)

Categorías en 'SCENARIO': ['AI_forecast' 'actual']


¿ Número de acutals y AI_forecast ?

In [ ]:
# Contar los registros de escenario
ActualsForecast = data['SCENARIO'].value_counts()

# Calcular porcentajes
total = ActualsForecast.sum()
porcentajes = (ActualsForecast / total) * 100

# Gráfico de Barras
fig = px.bar(porcentajes,
             x=porcentajes.values,
             y=porcentajes.index,
             orientation='h', # Barras en horizontal
             labels={'y': 'Escenario', 'x': 'Porcentaje (%)'},
             title='Distribución de Actual vs AI_forecast en Porcentajes',
             text=porcentajes.values  # Valores en las barras
)

# Truncar los porcentajes y poner %
fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside')

# Mostrar el gráfico
fig.show()



### **Varible 'COUNTRY'**

¿ Número de Países ?

In [ ]:
# Obtener el número de países únicos
paises = data['COUNTRY'].nunique()

# Mostrar el resultado
print(f"El número de países es: {paises}")


El número de países es: 9


In [ ]:
# Contar los registros de COUNTRY
VacesPais = data['COUNTRY'].value_counts()

# Calcular porcentajes
totalPais = VacesPais.sum()
porcentajesPais = (VacesPais / totalPais) * 100

# Gráfico de Pastel
fig = px.pie(
    names=porcentajesPais.index,
    values=porcentajesPais.values,
    title='Distribución de veces que aparece cada país',
    labels={'values': 'Porcentaje (%)', 'names': 'País'},
    hole=0.3, # Agujero en el medio
    color_discrete_sequence=px.colors.qualitative.Set3  # Colores diferentes
)

# Mostrar porcentajes
fig.update_traces(textinfo='percent+label')

# Mostrar el gráfico
fig.show()

### **Variable 'SUBBRAND'**

¿ Número de Productos ?

In [ ]:
# Obtener el número de productos únicos
productos = data['SUBBRAND'].nunique()

# Mostrar el resultado
print(f"El número de productos diferentes es: {productos}")


El número de productos diferentes es: 6


In [ ]:
# Contar los registros de SUBBRAND
VacesProducto = data['SUBBRAND'].value_counts()

# Calcular porcentajes
totalProducto = VacesProducto.sum()
porcentajesProducto = (VacesProducto / totalProducto) * 100

# Gráfico de Pastel
fig = px.pie(
    names=porcentajesProducto.index,
    values=porcentajesProducto.values,
    title='Distribución de veces que aparece cada país',
    labels={'values': 'Porcentaje (%)', 'names': 'País'},
    hole=0.3,  # Agujero en el medio
    color_discrete_sequence=px.colors.qualitative.Set3  # Colores diferentes
)

# Mostrar porcentajes
fig.update_traces(textinfo='percent+label')

# Mostrar el gráfico
fig.show()

### **Variable 'FORECAST'**

In [ ]:
# Obtener el número de forecasts distintos
numero_forecasts_distintos = data['FORECAST'].nunique()

# Mostrar el resultado
print(f"El número de forecasts distintos es: {numero_forecasts_distintos}")

El número de forecasts distintos es: 12


### **Histórico de Datos: Actuals/Forecast**

In [ ]:
# Filtrar los datos para "actuals" y "AI_forecast"
dataActualsForecast = data[(data['SCENARIO'].isin(['actual', 'AI_forecast']))]

# Crear la columna de fecha
dataActualsForecast['DATE'] = pd.to_datetime(dataActualsForecast[['YEAR', 'MONTH']].assign(DAY=1))

# Agrupar por fecha y escenario, sumando el amount
aggregated_data = dataActualsForecast.groupby(['DATE', 'SCENARIO'], as_index=False)['AMOUNT'].sum()

# Graficar
fig = px.line(aggregated_data, x='DATE', y='AMOUNT',
              color='SCENARIO',
              title='Histórico de Datos: Actuals vs Forecast',
              labels={'AMOUNT': 'Cantidad', 'DATE': 'Fecha'})

# Mostrar el gráfico
fig.show()


### **Horizontes de Previsión:**

In [ ]:
# Contar las frecuencias de tamaños directamente usando groupby
frecuencias = data.groupby(['COUNTRY', 'SUBBRAND', 'FORECAST']).size().value_counts().reset_index()

# Renombrar las columnas
frecuencias.columns = ['Tamaño', 'Frecuencia']

# Gráfico de Pastel
fig = px.pie(
    frecuencias,
    names='Tamaño',
    values='Frecuencia',
    title='Tamaños de Horizontes de Previsión',
    hole=0.3,  # Agujero en el medio
    color_discrete_sequence=px.colors.qualitative.Set3
)
fig.update_traces(textinfo='percent+label')

# Mostrar el gráfico
fig.show()



## 1. Distribución de las ventas Realizadas:



### a) Por Países:

In [ ]:
# Agrupar por país y escenario, sumando las ventas
ventasPais = data.groupby(['COUNTRY', 'SCENARIO'])['AMOUNT'].sum().reset_index()

# Calcular las ventas totales por país
ventasPaisTotales = ventasPais.groupby('COUNTRY')['AMOUNT'].sum().reset_index()

# Ordenar los países de mayor a menor según las ventas totales
ventasPaisTotales = ventasPaisTotales.sort_values(by='AMOUNT', ascending=False)

# Unir las ventas por país y escenario con las ventas totales ordenadas
ventasPais['COUNTRY'] = pd.Categorical(ventasPais['COUNTRY'], categories=ventasPaisTotales['COUNTRY'], ordered=True)
ventasPais = ventasPais.sort_values('COUNTRY')

fig = px.bar(ventasPais,
             x='COUNTRY',
             y='AMOUNT',
             color='SCENARIO',
             title='Distribución de Ventas por País y Escenario',
             labels={'COUNTRY': 'País', 'AMOUNT': 'Ventas Totales'},
             text_auto=True
)

# Modo de Gráfico de Barras Agrupado
fig.update_layout(barmode='group')

# Hacemos que el texto se vea fuera de las barras
fig.for_each_trace(lambda t: t.update(textposition='outside'))

# Mostrar el gráfico
fig.show()


### b) Por mes y año:

In [ ]:
data['DATE'] = pd.to_datetime(data[['YEAR', 'MONTH']].assign(DAY=1))

# Agrupamos por mes y año y sumamos las ventas
ventas_por_mes = data.groupby(data['DATE'].dt.to_period('M'))['AMOUNT'].sum().reset_index()

# Convertir a formato de fecha
ventas_por_mes['DATE'] = ventas_por_mes['DATE'].dt.to_timestamp()

# Gráfico de Líneas
fig = px.line(ventas_por_mes, x='DATE', y='AMOUNT',
              title='Distribución de Ventas por Mes y Año (Totales)',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True
)

fig.show()


In [ ]:
data['DATE'] = pd.to_datetime(data[['YEAR', 'MONTH']].assign(DAY=1))

# Agrupamos por país, mes y año y sumamos las ventas
ventas_por_mes = data.groupby(['COUNTRY', data['DATE'].dt.to_period('M')])['AMOUNT'].sum().reset_index()

# Convertir a formato de fecha
ventas_por_mes['DATE'] = ventas_por_mes['DATE'].dt.to_timestamp()

# Gráfico de Área
fig = px.area(ventas_por_mes, x='DATE', y='AMOUNT',
              title='Distribución de Ventas por Mes y Año por País',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              color='COUNTRY'
)

fig.show()

### c) Por marca:

In [ ]:
# Agrupamos por submarca y sumamos las ventas
ventas_por_submarca = data.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()

# Gráfico de Pastel
fig = px.pie(ventas_por_submarca, values='AMOUNT', names='SUBBRAND',
             title='Distribución de Ventas por Marca',
             labels={'SUBBRAND': 'Marca', 'AMOUNT': 'Ventas Totales'},
             hole = 0.3,
             color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(textinfo='percent+label')

# Mostrar el gráfico
fig.show()

## 2. Tendencia y Estacionalidad:

### a) Todas las ventas del país con menos ventas

País con menos ventas:

In [ ]:
# Agrupamos por país y sumamos las ventas
ventasPais = data.groupby('COUNTRY')['AMOUNT'].sum().reset_index()

# Encontramos el país con menos ventas
paisMenosVentas = ventasPais.loc[ventasPais['AMOUNT'].idxmin()]
print(paisMenosVentas)


COUNTRY               Spain
AMOUNT     218493569.403429
Name: 8, dtype: object


In [ ]:
# Agrupamos por país y sumamos las ventas
ventasPais = data.groupby('COUNTRY')['AMOUNT'].sum().reset_index()

# Encontrar el país con menos ventas
paisMenosVentas = ventasPais.loc[ventasPais['AMOUNT'].idxmin()]

# Filtrar datos para el país con menos ventas y escenario 'actual'
datosPais = data[(data['COUNTRY'] == paisMenosVentas['COUNTRY']) & (data['SCENARIO'] == 'actual')]

datosPais['DATE'] = pd.to_datetime(datosPais[['YEAR', 'MONTH']].assign(DAY=1))
ventasFecha = datosPais.groupby('DATE')['AMOUNT'].sum().reset_index()

# Gráfico de Líneas
fig = px.line(ventasFecha, x='DATE', y='AMOUNT',
              title=f'Tendencia de Ventas en {paisMenosVentas["COUNTRY"]}: Ventas Reales',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True
)

fig.show()

<ipython-input-20-dd4bcb6605db>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# Agrupamos por país y sumamos las ventas
ventasPais = data.groupby('COUNTRY')['AMOUNT'].sum().reset_index()

# Encontrar el país con menos ventas
paisMenosVentas = ventasPais.loc[ventasPais['AMOUNT'].idxmin()]

# Filtrar datos para el país con menos ventas y ambos escenarios 'actual' y 'AI_forecast'
datosPais = data[(data['COUNTRY'] == paisMenosVentas['COUNTRY']) & (data['SCENARIO'].isin(['actual', 'AI_forecast']))]

# Crear columna 'DATE'
datosPais['DATE'] = pd.to_datetime(datosPais[['YEAR', 'MONTH']].assign(DAY=1))

# Agrupar por fecha y escenario, sumando las ventas
ventasFecha = datosPais.groupby(['DATE', 'SCENARIO'])['AMOUNT'].sum().reset_index()

# Gráfico de Líneas con ambas categorías
fig = px.line(ventasFecha, x='DATE', y='AMOUNT', color='SCENARIO',
              title=f'Tendencia de Ventas en {paisMenosVentas["COUNTRY"]}: Actual y AI_forecast',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True
)

# Mostrar el gráfico
fig.show()

<ipython-input-21-d6d7298b2e33>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Podemos observar que los valores de predicción son mucho mayores que los valores reales. Por lo tanto, vamos a estudiar por qué pasa esto.





In [49]:
# Filtrar los datos para el país Spain, el escenario AI_forecast, y una submarca específica
datosSpainAI = data[(data['COUNTRY'] == 'Spain') &
                    # (data['SCENARIO'] == 'AI_forecast') &
                    (data['FORECAST'] == 'AI_P10F') &
                    (data['SUBBRAND'] == 'Pepsi Regular (L3)')]

# Crear columna 'DATE' para mejor organización de los datos
datosSpainAI['DATE'] = pd.to_datetime(datosSpainAI[['YEAR', 'MONTH']].assign(DAY=1))

# Ordenar por fecha
datosSpainAI = datosSpainAI.sort_values(by='DATE')

# Mostrar la tabla
print(len(datosSpainAI))
datosSpainAI.head()

54


<ipython-input-49-26709a637958>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT,DATE
6798,Spain,Pepsi Regular (L3),2023,10,AI_forecast,AI_P10F,2023.0,153166.298991,2023-10-01
13733,Spain,Pepsi Regular (L3),2023,10,AI_forecast,AI_P10F,2023.0,144399.950702,2023-10-01
2062,Spain,Pepsi Regular (L3),2023,10,AI_forecast,AI_P10F,2023.0,142858.058146,2023-10-01
15074,Spain,Pepsi Regular (L3),2023,11,AI_forecast,AI_P10F,2023.0,127347.454712,2023-11-01
6572,Spain,Pepsi Regular (L3),2023,11,AI_forecast,AI_P10F,2023.0,141309.803551,2023-11-01


Obersavamos que hay **54 datos**, para **18 meses**, por lo que 3 cantidades diferentes para cada mes. Vamos a quedarnos con la primera categoria de 'AMOUNT' de cada mes.

In [50]:
# Eliminar duplicados, quedándote solo con el primer registro de cada mes
datosSpainAI1 = datosSpainAI.drop_duplicates(subset='DATE', keep='first')

# Mostrar la tabla
print(len(datosSpainAI1))

18


In [53]:
# Agrupamos por país y sumamos las ventas
ventasPais = data.groupby('COUNTRY')['AMOUNT'].sum().reset_index()

# Encontrar el país con menos ventas
paisMenosVentas = ventasPais.loc[ventasPais['AMOUNT'].idxmin()]

# Filtrar datos para el país con menos ventas y escenario 'actual'
datosPais = data[(data['COUNTRY'] == paisMenosVentas['COUNTRY']) & (data['SCENARIO'] == 'actual')]

datosPais['DATE'] = pd.to_datetime(datosPais[['YEAR', 'MONTH']].assign(DAY=1))
ventasFecha = datosPais.groupby('DATE')['AMOUNT'].sum().reset_index()

# Gráfico de Líneas para ventas reales
fig = px.line(ventasFecha, x='DATE', y='AMOUNT',
              title=f'Tendencia de Ventas en {paisMenosVentas["COUNTRY"]}: Ventas Reales vs AI Forecast',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Ventas Totales'},
              markers=True
)

# Añadir la segunda serie de datos de AI_forecast con SUBBRAND específico
fig.add_scatter(x=datosSpainAI1['DATE'], y=datosSpainAI1['AMOUNT'],
                mode='lines+markers', name='AI_forecast')

# Mostrar el gráfico con ambas series
fig.show()


<ipython-input-53-3be59710cdda>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [52]:
# Extraer año y mes para la estacionalidad
ventasFecha['YEAR'] = ventasFecha['DATE'].dt.year
ventasFecha['MONTH'] = ventasFecha['DATE'].dt.month

# Agrupar por mes y calcular el promedio de ventas por mes
ventas_estacionalidad = ventasFecha.groupby('MONTH')['AMOUNT'].mean().reset_index()

# Gráfico de Barras para visualizar la estacionalidad
fig_estacionalidad = px.bar(ventas_estacionalidad, x='MONTH', y='AMOUNT',
                             title=f'Estacionalidad de Ventas en {paisMenosVentas["COUNTRY"]}',
                             labels={'MONTH': 'Mes', 'AMOUNT': 'Ventas Promedio'},
                             text='AMOUNT')

fig_estacionalidad.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig_estacionalidad.show()


### b) Marca más vendida: